In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import DeepIsolationForestTS as model
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal,total_list = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [4]:
clf_imu = model(epochs=50,batch_size=8,seq_len=1,stride=1)
clf_imu.fit(train_imu)
clf_audio = model(epochs=50,batch_size=8,seq_len=1,stride=1)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = model(epochs=50,batch_size=8,seq_len=1,stride=1)
clf_all.fit(train_combine)

Start Training...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:08<00:00,  5.81it/s]


Start Inference on the training data...
Start Inference...


100%|██████████| 50/50 [00:10<00:00,  4.91it/s]


...

100%|██████████| 50/50 [00:19<00:00,  2.55it/s]


Start Training...


100%|██████████| 50/50 [00:19<00:00,  2.60it/s]


Start Inference on the training data...
Start Inference...


100%|██████████| 50/50 [00:20<00:00,  2.42it/s]


In [5]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all        = clf_all.decision_function(test_combine)

Start Inference...


100%|██████████| 50/50 [00:07<00:00,  6.34it/s]


Start Inference...


100%|██████████| 50/50 [00:04<00:00, 11.54it/s]


Start Inference...


100%|██████████| 50/50 [00:08<00:00,  6.08it/s]


In [6]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu+scores_audio)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio[:-2])
print(result_imu[:-2])
print(result_all[:-2])

# np.save("deepforest_P",result_audio[-2])
# np.save("deepforest_r",result_audio[-1])

(0.9466176983652311, 0.9233719219354152, 0.8804433197711293, 0.8685503685503686, 0.8926767676767676)
(0.6577296401515151, 0.5970790641480203, 0.575785714541593, 0.5067178502879078, 0.6666666666666666)
(0.5353068098750664, 0.4531145667284672, 0.565710228763788, 0.3944223107569721, 1.0)


In [7]:
from sklearn import metrics
precision, recall, thresholds= metrics.precision_recall_curve(labels_all, (scores_audio))
epsilon = 1e-10

f1_scores = 2 * (precision * recall) / (precision + recall + epsilon)
best_index = np.argmax(f1_scores)
best_threshold = thresholds[best_index]
print(best_threshold)
indoor_group = []
outdoor_group = []
mach_group = []
coli_group = []
for idx, filepath in enumerate(total_list):
    parts = filepath.split('/')
    sequence = parts[-2]  # e.g., 'iseq2', 'iseq5', 'seq16'
    if sequence=='seq16':
        file_number = int(parts[-1].split('.')[0])  # Extract file number from filename (e.g., '0' from '0.npy')

    # Check if the file belongs to the indoor group (iseq2, iseq5, or seq16 with 0-104.npy)
    if sequence in ['iseq2', 'iseq5'] or (sequence == 'seq16' and  file_number >= 104):
        indoor_group.append(idx)  # Add the index of the file
    elif sequence in ['seq15', 'seq10'] or (sequence == 'seq16' and  file_number < 104):
        outdoor_group.append(idx)  # Add the index of the file
    elif sequence in ['bl', 'br','fl','fr']:
        mach_group.append(idx)  # Add the index of the file
    else:
        coli_group.append(idx)  # Add the index of the file

indoor_losses = [scores_audio[idx] for idx in indoor_group]
outdoor_losses = [scores_audio[idx] for idx in outdoor_group]
mach_losses = [scores_audio[idx] for idx in mach_group]
coli_losses = [scores_audio[idx] for idx in coli_group]

1225.9994172460322


In [9]:
indoor_group = []
outdoor_group = []
mach_group = []
coli_group = []
for idx, filepath in enumerate(total_list):
    parts = filepath.split('/')
    sequence = parts[-2]  # e.g., 'iseq2', 'iseq5', 'seq16'
    if sequence=='seq16':
        file_number = int(parts[-1].split('.')[0])  # Extract file number from filename (e.g., '0' from '0.npy')

    # Check if the file belongs to the indoor group (iseq2, iseq5, or seq16 with 0-104.npy)
    if sequence in ['iseq2', 'iseq5'] or (sequence == 'seq16' and  file_number >= 104):
        indoor_group.append(idx)  # Add the index of the file
    elif sequence in ['seq15', 'seq10'] or (sequence == 'seq16' and  file_number < 104):
        outdoor_group.append(idx)  # Add the index of the file
    elif sequence in ['bl', 'br','fl','fr']:
        mach_group.append(idx)  # Add the index of the file
    else:
        coli_group.append(idx)  # Add the index of the file

indoor_losses = [scores_audio[idx] for idx in indoor_group]
outdoor_losses = [scores_audio[idx] for idx in outdoor_group]
mach_losses = [scores_audio[idx] for idx in mach_group]
coli_losses = [scores_audio[idx] for idx in coli_group]

In [10]:
# Given threshold
threshold = best_threshold
# Example threshold, replace with actual value

# True labels for indoor and outdoor groups
true_labels_indoor = [0] * len(indoor_losses)  # All indoor samples are normal
true_labels_outdoor = [0] * len(outdoor_losses)  # All outdoor samples are anomalies

# Combine losses and true labels
all_losses = indoor_losses + outdoor_losses
all_true_labels = true_labels_indoor + true_labels_outdoor

# Calculate predicted labels based on the threshold
predicted_labels = [1 if score > threshold else 0 for score in all_losses]

# Calculate accuracy
correct_predictions = sum(pred == true for pred, true in zip(predicted_labels, all_true_labels))
total_samples = len(all_true_labels)
accuracy = correct_predictions / total_samples if total_samples > 0 else 0

# Output accuracy for both groups
print("Overall Accuracy:")
print(f"  Accuracy: {accuracy:.4f}")

# Optional: Calculate accuracy separately for indoor and outdoor groups
accuracy_indoor = sum(predicted_labels[i] == true_labels_indoor[i] for i in range(len(indoor_losses))) / len(indoor_losses) if len(indoor_losses) > 0 else 0
accuracy_outdoor = sum(predicted_labels[len(indoor_losses) + i] == true_labels_outdoor[i] for i in range(len(outdoor_losses))) / len(outdoor_losses) if len(outdoor_losses) > 0 else 0

print("\nIndoor Group Accuracy:")
print(f"  Accuracy: {accuracy_indoor:.4f}")

print("\nOutdoor Group Accuracy:")
print(f"  Accuracy: {accuracy_outdoor:.4f}")


Overall Accuracy:
  Accuracy: 0.9120

Indoor Group Accuracy:
  Accuracy: 0.8894

Outdoor Group Accuracy:
  Accuracy: 0.9373


In [11]:
true_labels_mach = [1] * len(mach_losses)  # All indoor samples are normal
true_labels_coli = [1] * len(coli_losses)  # All outdoor samples are anomalies

# Combine losses and true labels
all_losses = mach_losses + coli_losses
all_true_labels = true_labels_mach + true_labels_coli

# Calculate predicted labels based on the threshold
predicted_labels = [1 if score > threshold else 0 for score in all_losses]

# Calculate accuracy
correct_predictions = sum(pred == true for pred, true in zip(predicted_labels, all_true_labels))
total_samples = len(all_true_labels)
accuracy = correct_predictions / total_samples if total_samples > 0 else 0

# Output accuracy for both groups
print("Overall Accuracy:")
print(f"  Accuracy: {accuracy:.4f}")

# Optional: Calculate accuracy separately for indoor and outdoor groups
accuracy_mach = sum(predicted_labels[i] == true_labels_mach[i] for i in range(len(mach_losses))) / len(mach_losses) if len(mach_losses) > 0 else 0
accuracy_coli = sum(predicted_labels[len(mach_losses) + i] == true_labels_coli[i] for i in range(len(coli_losses))) / len(coli_losses) if len(outdoor_losses) > 0 else 0

print("\nmach Group Accuracy:")
print(f"  Accuracy: {accuracy_mach:.4f}")

print("\ncoli Group Accuracy:")
print(f"  Accuracy: {accuracy_coli:.4f}")

Overall Accuracy:
  Accuracy: 0.8914

mach Group Accuracy:
  Accuracy: 0.9552

coli Group Accuracy:
  Accuracy: 0.7742
